In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_columns',100)

In [3]:
df=pd.read_csv('declinestats.csv')

In [4]:
df1=pd.read_csv('cleanData.csv')

In [5]:
newData=df1[['loan_amnt','title','fico_range_low','dti','zip_code','addr_state','emp_length','policy_code','set','timestamp']]

In [6]:
newData=newData.append(df,ignore_index=True)

In [7]:
newData['yStatus'] = newData['set'].apply(lambda x: 1 if x == "accepted" else 0)

In [8]:
newData.to_csv('classificationData.csv')

In [9]:
def elength(a):
    if (a=='n/a'):
        return 0
    elif (a=='10+ years'):
        return 10
    elif (a=='1 year'):
        return 1
    elif (a=='< 1 year'):
        return 0.5
    else:
        return float(a.strip(' years'))
    
newData['yemp_length'] = newData['emp_length'].map(lambda a: elength(a))

In [10]:
colsToKeep=['loan_amnt','fico_range_low','dti','yemp_length','policy_code']

In [11]:
X=newData[colsToKeep]

In [12]:
y=newData['yStatus']

In [13]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=60)

In [15]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
y_pred = model.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[3226328       0]
 [      0  302485]]


In [18]:
#Compute FPR and TPR
from sklearn.metrics import roc_curve
preds = model.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, preds)

In [9]:
#Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.1])
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

NameError: name 'fpr' is not defined